In [1]:
import gammapy 
print(f'loaded gammapy version: {gammapy.__version__} ' )
print(f'Supposed to be 1.0 (21-12-2022)' )

loaded gammapy version: 0.1.dev19174+g56b5bdf 
Supposed to be 1.0 (21-12-2022)


In [2]:
#get_ipython().system('jupyter nbconvert --to script 1-Nui_Par_Fitting.ipynb')
import pyximport

pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import gammapy

# from gammapy.datasets import MapDataset
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit, Parameter, Parameters, Covariance
from gammapy.datasets import MapDataset #, MapDatasetNuisance
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    create_crab_spectral_model,
    SkyModel,
    PointSpatialModel,
    ShellSpatialModel,
    GeneralizedGaussianSpatialModel,
    TemplateSpatialModel,
    LogParabolaSpectralModel,
    GaussianSpatialModel,
    DiskSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.maps import MapAxis
from gammapy.modeling.models.spectral import scale_plot_flux
from gammapy.estimators import  FluxPointsEstimator
    
from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('/home/katrin/Documents/nuisance_summary/')
sys.path.append('../')
from Dataset_Creation import sys_dataset
import Dataset_load 

## Read dataset

In [3]:
scaled_amplitude = Parameter('amplitude',
                             value = 1e-12)
dataset_asimov  = Dataset_load.create_asimov(model = 'pl', source = "PKSflare",
                                            parameters = Parameters([scaled_amplitude]))

loaded dataset:
/home/wecapstor1/caph/mppi045h/nuisance_summary/PKS_flare/HESS_public/dataset-simulated-pl.fits.gz


## Varying the Exposure

In [4]:
mask = dataset_asimov.mask.data.sum(axis=2).sum(axis=1)>0

ebins = dataset_asimov.counts.geom.axes[0].center[mask]
print(len(ebins))

20


In [5]:
shift = +0.1
tilt = 0.02


N = 300
sigma_a = shift
sigma_i = tilt

save = True
save_flux = True




In [ ]:
for n in range(N):
    print()
    print('====' * 30)
    print(n)
    print('====' * 30)
    
    shift_rnd = np.random.normal(0, shift, 1)
    tilt_rnd = np.random.normal(0, tilt, 1)
    print(f"shift:, {shift_rnd}, tilt: {tilt_rnd}" )
    sys_d_cor = sys_dataset(dataset_asimov= dataset_asimov,
                    shift = shift_rnd, 
                    tilt = tilt_rnd,
                    rnd = True)
    dataset = sys_d_cor.create_dataset()
    fit_cor = Fit(store_trace=False)
    minuit_opts = {"tol": 0.001, "strategy": 2}
    fit_cor.optimize_opts = minuit_opts
    result_cor = fit_cor.run([dataset])


    if save:
        with open("data/7a_P_draw_info.txt", "a") as myfile:
            myfile.write(str(float(shift_rnd)) + '    '+ str(float(tilt_rnd)) + '    ' +  str(float(dataset.stat_sum())) + '\n')

    stri = ""
    for p in ['amplitude', 'index', 'norm', 'tilt']:
        stri += str(dataset.models.parameters[p].value)  + '   ' +  str(dataset.models.parameters[p].error)  + '   '
    print(stri)
    if save:
        with open("data/7a_P_draw_par.txt", "a") as myfile:
            myfile.write(stri + '\n')



    fluxes = []
    for e in ebins:
        flux =  dataset.models[0].spectral_model(e)
        fluxes.append(flux.value)

    ff = str()
    for f in fluxes:
        ff += str(f) + "  "
    #print(ff)
    if save:
        with open("data/7a_P_draw_flux.txt", "a") as myfile:
            myfile.write( ff + '\n')


    energy_bounds = (ebins[0], ebins[-1] ) * u.TeV

    energy_min, energy_max = energy_bounds
    energy = MapAxis.from_energy_bounds(
        energy_min,
        energy_max,
        len(ebins),
    )

    fluxe2, _ = dataset.models[0].spectral_model._get_plot_flux(sed_type='dnde', energy=energy)
    fluxe2 = scale_plot_flux(fluxe2, energy_power=2)
    fluxe2 = fluxe2.quantity[:, 0, 0]
    fluxe2 = np.array(fluxe2)   
    ff = str()
    for f in fluxe2:
        ff += str(f) + "  "
    if save:
        with open("data/7a_P_draw_flux2e.txt", "a") as myfile:
            myfile.write( ff + '\n')

    energy_edges = dataset.geoms['geom'].axes[0].edges
    #esti  = FluxPointsEstimator(energy_edges= energy_edges)
    #fluxpoints = esti.run([dataset])
    #if save_flux:
    #    fluxpoints.write(f'data/fluxpoints/6_fluxpoints_{shift_rnd[0]:.6}.fits')
    #except:
    #    pass



0
shift:, [0.08668065], tilt: [-0.02642663]
1.0055297700004264e-12   4.7083858183642053e-14   2.366834920786533   0.04544885824771994   0.9990786673660769   0.0015597281499847267   0.0   0.0   

1
shift:, [-0.12282532], tilt: [-0.00446901]
1.0618419393869482e-12   5.703180504489969e-14   2.2832798485338444   0.04865682828909057   0.9982959985242236   0.0015590013991573647   0.0   0.0   

2
shift:, [-0.03224508], tilt: [0.01089325]
9.891965229292092e-13   5.2295637270714045e-14   2.257477296092084   0.04743609100499187   1.0010235482598093   0.0015612057486642033   0.0   0.0   

3
shift:, [0.12159569], tilt: [0.00795349]
8.280232141503126e-13   4.5198000753028224e-14   2.2282595145996225   0.04741900466538091   0.9990351574803036   0.0015596087109009886   0.0   0.0   

4
shift:, [-0.0461918], tilt: [0.03399113]
1.0628388288520766e-12   5.314082128233816e-14   2.266626715398767   0.04637142284436053   0.9973667985568049   0.0015583192806574325   0.0   0.0   

5
shift:, [0.03800882], til